# Find Kth Largest Value in BST

Write a function that takes in a **Binary Search Tree (BST)** and a positive integer **k** and return the kth largest integer contained in the **BST**.

You can assume that there will only be integer values in the BST and that **k** is less than or equal to the number of nodes in the tree.

Also, for the purpose of this question, duplicate integers will be treated as distinct values. In other words, the second largest value in a BST containing values `{5,7,7}`will be `7`- not `5`.

Each **BST** node has an integer **value**, a **left** child node, and a **right** child node. A node is said to be a valid **BST** node if and only if it satisfies the **BST** property:

- its **value** is strictly greater than the values of every onde to its left;
- its **value** is less than or equal to the values of every node to is rights;
- its children nodes are either valid **BST** nodes themselves or **None**/**null**.

Sample Input

```python
 tree =   10
       /     \
      5       20
    /   \   /   \
   2    5  17    22
  / \          
 1   3        

k = 3
```

Sample output

```python
17
```

**Hints**

1. Make sure to consider the fact that the given tree is a Binary Search Tree - not just a regular Binary Tree. How does this fact help you the problem in a more optimal time complexity?

2. The brute-force approach to this problem is to simply perform an in-order traversal of this **BST** and to store all of its node's values in the order in which they're visited. Since an in-order traversal of a **BST** visits the nodes in ascending order, the ``k``th value from the end of the traversal order will be the `k`th largest value.

3. You can actually solve this problem in `O(h + k)` time, where `h` is the height of the tree. Rather than looking at the nodes in ascending order, you shold look at them in descending order.

4. To solve this problem in `O(h + k)`time as mentioned in Hint 3, you need to perform a **reverse** in-order traversal. Since you'll be looking at nodes in descending order, you can simply return the `k`th visited in the reverse in-order traversal.

**Optimal Space & Time Complexity**

- O(h+k) time | O(h) space - where `h`is the height of the tree and `k`is the input parameter.

In [1]:
%%file binarysearchtree.py
import plotly.graph_objs as go

class Node:
    """
    A class representing a node in a binary search tree.

    Attributes:
    - value: the value of the node
    - left_child: the left child of the node
    - right_child: the right child of the node
    """

    def __init__(self, value):
        """
        Initializes a new instance of the Node class.

        Args:
        - value: the value of the node
        """
        self.value = value
        self.left_child = None
        self.right_child = None


class BST:
    """
    A class representing a binary search tree.

    Attributes:
    - root: the root node of the tree
    """

    def __init__(self):
        """
        Initializes a new instance of the BST class.
        """
        self.root = None

    def add(self, value):
        """
        Adds a new node with the given value to the tree.

        Args:
        - value: the value of the node to add
        """
        if self.root is None:
            # The root does exist yet, create it
            self.root = Node(value)
        else:
            # Find the right place and insert new value
            self._add_recursive(self.root, value)

    def _add_recursive(self, current_node, value):
        """
        A helper method to recursively traverse the tree and find the correct position to add the new node.

        Args:
        - current_node: the current node to traverse
        - value: the value of the node to add
        """
        if value <= current_node.value:
            # Go to the left
            if current_node.left_child is None:
                current_node.left_child = Node(value)
            else:
                self._add_recursive(current_node.left_child, value)
        else:
            # Go to the right
            if current_node.right_child is None:
                current_node.right_child = Node(value)
            else:
                self._add_recursive(current_node.right_child, value)

    def _contains(self, current_node, value):
        """
        A helper method to recursively traverse the tree and find the node with the given value.

        Args:
        - current_node: the current node to traverse
        - value: the value to search for

        Returns:
        - True if a node with the given value is found, False otherwise
        """
        if current_node is None:
            return False
        if current_node.value == value:
            return True
        if value < current_node.value:
            return self._contains(current_node.left_child, value)
        return self._contains(current_node.right_child, value)

    def contains(self, value):
        """
        Checks whether a node with the given value is present in the tree.

        Args:
        - value: the value to search for

        Returns:
        - True if a node with the given value is found, False otherwise
        """
        return self._contains(self.root, value)

    def plot(self):
        """
        Plots the binary search tree using Plotly.
        """
        if self.root is None:
            print("The tree is empty!")
            return

        # Initialize lists for coordinates and connections
        node_coords = []
        lines = []

        # Helper function to traverse the tree and fill the coordinate and connection lists
        def _plot_recursive(node, x, y, offset):
            if node is not None:
                node_coords.append((x, y, node.value))
                if node.left_child is not None:
                    new_x = x - offset
                    new_y = y - 1
                    lines.append((x, y, new_x, new_y))
                    _plot_recursive(node.left_child, new_x, new_y, offset / 2)
                if node.right_child is not None:
                    new_x = x + offset
                    new_y = y - 1
                    lines.append((x, y, new_x, new_y))
                    _plot_recursive(node.right_child, new_x, new_y, offset / 2)

        # Traverse the tree starting from the root node
        _plot_recursive(self.root, x=0, y=0, offset=0.5)

        # Create a scatter plot for the nodes
        node_trace = go.Scatter(x=[x for x, y, _ in node_coords],
                                y=[y for _, y, _ in node_coords],
                                text=[str(val) for _, _, val in node_coords],
                                mode='markers+text',
                                textposition='top center',
                                marker=dict(symbol='circle',
                                            size=20,
                                            color='darkblue'))

        # Create a scatter plot for the connections between nodes
        line_trace = go.Scatter(x=sum([[x1, x2, None] for x1, y1, x2, y2 in lines], []),
                                y=sum([[y1, y2, None] for x1, y1, x2, y2 in lines], []),
                                mode='lines',
                                line=dict(color='black'))

        # Combine the two scatter plots
        layout = go.Layout(title='',
                           xaxis=dict(title='', showgrid=False, zeroline=False, showticklabels=False),
                           yaxis=dict(title='', showgrid=False, zeroline=False, showticklabels=False),
                           showlegend=False)

        fig = go.Figure(data=[node_trace, line_trace], layout=layout)
        fig.show()

Writing binarysearchtree.py


In [2]:
%run -i binarysearchtree.py

In [3]:
from binarysearchtree import *
# Example usage:
bst = BST()
for value in [5, 3, 1, 0, 2, 4, 7, 6, 8]:
    bst.add(value)
bst.plot()

# `findKthLargestValue`

The `findKthLargestValue` function is responsible for finding the k-th largest integer in a Binary Search Tree (BST). It employs an in-order traversal approach to collect the values of the tree nodes into a sorted list and then determines the k-th largest value from that list. This function assumes that the BST contains only integer values and treats duplicate integers as distinct values.

## Parameters

- `tree` (BST): The Binary Search Tree in which the k-th largest value is to be found.
- `k` (int): A positive integer representing the desired position in the order of values.

## Return

- `int`: The k-th largest integer present in the BST. If the value of `k` exceeds the total number of elements in the BST, the function returns `None`.

## Algorithm

1. **In-order Traversal:**
   - The `inorderTraversal` function is a helper function that performs the in-order traversal of the BST, collecting the values of the nodes into a list `values`.
   - In-order traversal visits the nodes in the following order: left, current node, right, ensuring that the values are collected in ascending order.

2. **Value Collection:**
   - In-order traversal starts from the root node of the BST by calling `inorderTraversal(tree.root, values)`.
   - During traversal, the values of the nodes are added to the `values` list.

3. **Determining the k-th Largest Value:**
   - After collecting all the values of the BST into the `values` list, it is checked whether the value of `k` falls within the range of values encountered during traversal.
   - If `k` is valid, the k-th largest value from `values` is returned. Otherwise, `None` is returned.

In [4]:
%%file closestvalue.py
import pytest
from binarysearchtree import *

def findKthLargestValue(tree, k):
    def inorderTraversal(node, values):
        if node is None:  # If the node is None, return
            return

        # Call inorderTraversal on the right subtree
        inorderTraversal(node.right_child, values)

        # Append the value of the current node to the list of values
        values.append(node.value)

        # Call inorderTraversal on the left subtree
        inorderTraversal(node.left_child, values)

    # call inorderTraversal starting from the root node and populate the 'values' list
    values = []
    inorderTraversal(tree.root, values)


    if k <= len(values):  # Check if the desired position 'k' is within the range of values found during traversal
        return values[k - 1]
    else:
        return None


@pytest.fixture(scope="session")
def data():

    array = [[15,5,20,17,22,2,5,1,3],
             [5,4,6,3,7],
             [5],
             [20,15,25,10,19,21,30,22],
             [1,2,3,4,5],
             [10,8,6,4,2],
             [10,8,6,9,4,7,2,5,3],
             [99727,99,727],
             [15,5,20,17,22,24,23,25,2,5,1,3],
             [15,5,20,17,22,2,5,1,3],
             [15,5,20,17,22,2,5,1,3]
             ]
    return array

def test_1(data):
    bst = BST()
    for value in data[0]:
      bst.add(value)
    assert findKthLargestValue(bst, 3) == 17

def test_2(data):
    bst = BST()
    for value in data[1]:
      bst.add(value)
    assert findKthLargestValue(bst, 1) == 7

def test_3(data):
    bst = BST()
    for value in data[2]:
      bst.add(value)
    assert findKthLargestValue(bst, 1) == 5

def test_4(data):
    bst = BST()
    for value in data[3]:
      bst.add(value)
    assert findKthLargestValue(bst, 3) == 22

def test_5(data):
    bst = BST()
    for value in data[4]:
      bst.add(value)
    assert findKthLargestValue(bst, 5) == 1

def test_6(data):
    bst = BST()
    for value in data[5]:
      bst.add(value)
    assert findKthLargestValue(bst, 2) == 8

def test_7(data):
    bst = BST()
    for value in data[6]:
      bst.add(value)
    assert findKthLargestValue(bst, 5) == 6

def test_8(data):
    bst = BST()
    for value in data[7]:
      bst.add(value)
    assert findKthLargestValue(bst, 1) == 99727

def test_9(data):
    bst = BST()
    for value in data[8]:
      bst.add(value)
    assert findKthLargestValue(bst, 7) == 15

def test_10(data):
    bst = BST()
    for value in data[9]:
      bst.add(value)
    assert findKthLargestValue(bst, 5) == 5

def test_11(data):
    bst = BST()
    for value in data[10]:
      bst.add(value)
    assert findKthLargestValue(bst, 6) == 5

Writing closestvalue.py


In [5]:
!pytest closestvalue.py -vv

======================================= test session starts ========================================
platform linux -- Python 3.10.12, pytest-7.4.4, pluggy-1.4.0 -- /usr/bin/python3
cachedir: .pytest_cache
rootdir: /content
plugins: anyio-3.7.1
collected 11 items                                                                                 

closestvalue.py::test_1 PASSED                                                               [  9%]
closestvalue.py::test_2 PASSED                                                               [ 18%]
closestvalue.py::test_3 PASSED                                                               [ 27%]
closestvalue.py::test_4 PASSED                                                               [ 36%]
closestvalue.py::test_5 PASSED                                                               [ 45%]
closestvalue.py::test_6 PASSED                                                               [ 54%]
closestvalue.py::test_7 PASSED                        